# Amadeus

This notebook walks you through connecting LangChain to the `Amadeus` travel APIs.

This `Amadeus` toolkit allows agents to make decision when it comes to travel, especially searching and booking trips with flights.

To use this toolkit, you will need to have your Amadeus API keys ready, explained in the [Get started Amadeus Self-Service APIs](https://developers.amadeus.com/get-started/get-started-with-self-service-apis-335). Once you've received a AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET, you can input them as environmental variables below.

Note: Amadeus Self-Service APIs offers a test environment with [free limited data](https://amadeus4dev.github.io/developer-guides/test-data/). This allows developers to build and test their applications before deploying them to production. To access real-time data, you will need to [move to the production environment](https://amadeus4dev.github.io/developer-guides/API-Keys/moving-to-production/).

In [ ]:
%pip install --upgrade --quiet  amadeus > /dev/null

In [ ]:
%pip install -qU langchain-community

## Assign Environmental Variables

The toolkit will read the AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET environmental variables to authenticate the user, so you need to set them here. 

In [1]:
# Set environmental variables here
import os

os.environ["AMADEUS_CLIENT_ID"] = "CLIENT_ID"
os.environ["AMADEUS_CLIENT_SECRET"] = "CLIENT_SECRET"

# os.environ["AMADEUS_HOSTNAME"] = "production" or "test"

## Create the Amadeus Toolkit and Get Tools

To start, you need to create the toolkit, so you can access its tools later.

By default, `AmadeusToolkit` uses `ChatOpenAI` to identify airports closest to a given location. To use it, just set `OPENAI_API_KEY`.


In [3]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [4]:
from langchain_community.agent_toolkits.amadeus.toolkit import AmadeusToolkit

toolkit = AmadeusToolkit()
tools = toolkit.get_tools()

Alternatively, you can use any LLM supported by langchain, e.g. `HuggingFaceHub`. 

In [ ]:
from langchain_community.llms import HuggingFaceHub

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HF_API_TOKEN"

llm = HuggingFaceHub(
    repo_id="tiiuae/falcon-7b-instruct",
    model_kwargs={"temperature": 0.5, "max_length": 64},
)

toolkit_hf = AmadeusToolkit(llm=llm)

## Use Amadeus Toolkit within an Agent

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description_and_args
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI(temperature=0)

prompt = hub.pull("hwchase17/react-json")
agent = create_react_agent(
    llm,
    tools,
    prompt,
    tools_renderer=render_text_description_and_args,
    output_parser=ReActJsonSingleInputOutputParser(),
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [8]:
agent_executor.invoke({"input": "What is the name of the airport in Cali, Colombia?"})



> Entering new AgentExecutor chain...
I should use the closest_airport tool to find the airport in Cali, Colombia.
Action: closest_airport
Action Input: location= "Cali, Colombia"content='{\n  "iataCode": "CLO"\n}'The airport in Cali, Colombia is called CLO.
Final Answer: CLO

> Finished chain.


{'input': 'What is the name of the airport in Cali, Colombia?',
 'output': 'CLO'}

In [7]:
agent_executor.invoke(
    {
        "input": "What is the departure time of the cheapest flight on March 10, 2024 leaving Dallas, Texas before noon to Lincoln, Nebraska?"
    }
)



> Entering new AgentExecutor chain...
Question: What is the departure time of the cheapest flight on March 10, 2024 leaving Dallas, Texas before noon to Lincoln, Nebraska?
Thought: We need to find the closest airport to Dallas, Texas, and then search for the cheapest flight departing before noon on March 10, 2024, to Lincoln, Nebraska.
Action:
```
{
  "action": "closest_airport",
  "action_input": {
    "location": "Dallas, Texas"
  }
}
```content='{\n    "iataCode": "DFW"\n}'Now, we have the IATA code for Dallas, Texas. Next, we will search for the cheapest flight departing before noon on March 10, 2024, from Dallas (DFW) to Lincoln, Nebraska.

Action:
```
{
  "action": "single_flight_search",
  "action_input": {
    "originLocationCode": "DFW",
    "destinationLocationCode": "LNK",
    "departureDateTimeEarliest": "2024-03-10T00:00:00",
    "departureDateTimeLatest": "2024-03-10T12:00:00"
  }
}
```[{'price': {'total': '593.35', 'currency': 'EURO'}, 'segments': [{'departure': {'iata

{'input': 'What is the departure time of the cheapest flight on March 10, 2024 leaving Dallas, Texas before noon to Lincoln, Nebraska?',
 'output': 'The departure time of the cheapest flight on March 10, 2024, leaving Dallas, Texas before noon to Lincoln, Nebraska is 10:54 AM.'}

In [19]:
agent_executor.invoke(
    {
        "input": "At what time does earliest flight on March 10, 2024 leaving Dallas, Texas to Lincoln, Nebraska land in Nebraska?"
    }
)



> Entering new AgentExecutor chain...
Question: At what time does the earliest flight on March 10, 2024, leaving Dallas, Texas, to Lincoln, Nebraska land in Nebraska?

Thought: We need to find the closest airport to Dallas, Texas, and then search for a single flight from there to Lincoln, Nebraska on March 10, 2024.

Action:
```
{
  "action": "closest_airport",
  "action_input": {
    "location": "Dallas, Texas"
  }
}
```
content='{\n    "iataCode": "DFW"\n}'Now that we have the closest airport to Dallas, Texas, which is Dallas/Fort Worth International Airport with the IATA code DFW, we can proceed to search for a single flight from DFW to Lincoln, Nebraska on March 10, 2024.

Action:
```
{
  "action": "single_flight_search",
  "action_input": {
    "originLocationCode": "DFW",
    "destinationLocationCode": "LNK",
    "departureDateTimeEarliest": "2024-03-10T00:00:00",
    "departureDateTimeLatest": "2024-03-10T23:59:59"
  }
}
```
[{'price': {'total': '593.35', 'currency': 'EURO'}, 

{'input': 'At what time does earliest flight on March 10, 2024 leaving Dallas, Texas to Lincoln, Nebraska land in Nebraska?',
 'output': 'The earliest flight on March 10, 2024, leaving Dallas, Texas to Lincoln, Nebraska lands in Nebraska at 14:19 on March 11, 2024.'}

In [8]:
# to execute api correctly, change the querying date to feature
agent_executor.invoke(
    {
        "input": "What is the full travel time for the cheapest flight between Portland, Oregon to Dallas, TX on March 10, 2024?"
    }
)



> Entering new AgentExecutor chain...
Question: What is the full travel time for the cheapest flight between Portland, Oregon to Dallas, TX on March 10, 2024?
Thought: We need to find the cheapest flight between Portland, Oregon and Dallas, TX on March 10, 2024, and then calculate the total travel time.
Action:
```
{
  "action": "single_flight_search",
  "action_input": {
    "originLocationCode": "PDX",
    "destinationLocationCode": "DFW",
    "departureDateTimeEarliest": "2024-03-10T00:00:00",
    "departureDateTimeLatest": "2024-03-10T23:59:59"
  }
}
```[{'price': {'total': '246.13', 'currency': 'EURO'}, 'segments': [{'departure': {'iataCode': 'PDX', 'at': '2024-03-10T12:09:00'}, 'arrival': {'iataCode': 'LAS', 'terminal': '1', 'at': '2024-03-10T14:22:00'}, 'flightNumber': '427', 'carrier': 'SPIRIT AIRLINES'}, {'departure': {'iataCode': 'LAS', 'terminal': '1', 'at': '2024-03-11T05:00:00'}, 'arrival': {'iataCode': 'DFW', 'terminal': 'E', 'at': '2024-03-11T09:39:00'}, 'flightNumber'

 Error: Earliest and latest departure dates need to be the  same date. If you're trying to search for round-trip  flights, call this function for the outbound flight first,  and then call again for the return flight. 


We have retrieved the flight options, but we need to calculate the total travel time for each option to find the cheapest flight's full travel time.
Action:
```
{
  "action": "single_flight_search",
  "action_input": {
    "originLocationCode": "PDX",
    "destinationLocationCode": "DFW",
    "departureDateTimeEarliest": "2024-03-10T12:09:00",
    "departureDateTimeLatest": "2024-03-11T14:11:00"
  }
}
```[None]It seems that we are unable to retrieve the specific flight details for the cheapest option to calculate the full travel time. We can estimate the total travel time based on the layover and flight durations provided in the previous observations.

Final Answer: The full travel time for the cheapest flight between Portland, Oregon and Dallas, TX on March 10, 2024, is approximately 21 hours and 2 minutes.

> Finished chain.


{'input': 'What is the full travel time for the cheapest flight between Portland, Oregon to Dallas, TX on March 10, 2024?',
 'output': 'The full travel time for the cheapest flight between Portland, Oregon and Dallas, TX on March 10, 2024, is approximately 21 hours and 2 minutes.'}

In [11]:
agent_executor.invoke(
    {
        "input": "Please draft a concise email from Santiago to Paul, Santiago's travel agent, asking him to book the earliest flight from DFW to DCA on March 10, 2024. Include all flight details in the email."
    }
)



> Entering new AgentExecutor chain...
Question: Please draft a concise email from Santiago to Paul, Santiago's travel agent, asking him to book the earliest flight from DFW to DCA on March 10, 2024. Include all flight details in the email.

Thought: We need to find the earliest flight from Dallas Fort Worth (DFW) to Washington D.C. (DCA) on March 10, 2024, and provide all the necessary flight details in the email.

Action:
```
{
  "action": "single_flight_search",
  "action_input": {
    "originLocationCode": "DFW",
    "destinationLocationCode": "DCA",
    "departureDateTimeEarliest": "2024-03-10T00:00:00",
    "departureDateTimeLatest": "2024-03-10T23:59:59"
  }
}
```
[{'price': {'total': '303.31', 'currency': 'EURO'}, 'segments': [{'departure': {'iataCode': 'DFW', 'terminal': 'E', 'at': '2024-03-10T06:00:00'}, 'arrival': {'iataCode': 'EWR', 'terminal': 'C', 'at': '2024-03-10T10:25:00'}, 'flightNumber': '1517', 'carrier': 'UNITED AIRLINES'}, {'departure': {'iataCode': 'EWR', 'termi

{'input': "Please draft a concise email from Santiago to Paul, Santiago's travel agent, asking him to book the earliest flight from DFW to DCA on March 10, 2024. Include all flight details in the email.",
 'output': 'We have found several flight options from Dallas Fort Worth (DFW) to Washington D.C. (DCA) on March 10, 2024. The earliest flight is with United Airlines, departing from DFW at 06:00 and arriving at DCA at 13:19 with flight numbers 1517 and 4431. The total price is 303.31 EURO.'}